In [1]:
from matplotlib import pyplot as plt
import math
import numpy as np
import pandas as pd
import xarray as xr
import glob
from cmip6_preprocessing.preprocessing import combined_preprocessing
import cftime
from datetime import datetime
xr.set_options(display_style='html')
plt.rcParams['figure.figsize'] = 8,3

import gsw as gs
from IPython.display import clear_output

In [2]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

### We calculate density, buoyancy frequency, and isopycnal slope from T and S.
#### First read in temp and salinity data

In [3]:
data_dir = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip6/CONCATTED/'
data_dir_so = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_so/cmip6/'


files_c6 =  glob.glob(data_dir+ 'thetao_Oyr_*_piControl*.nc')
files_so_c6 =  glob.glob(data_dir_so+ 'so_Oyr_*_piControl*.nc')


models=[]
for file in files_c6:
    model = file.split("_")[5]
    models.append(model)
MODELS=np.unique(models)


models=[]
for file in files_so_c6:
    model = file.split("_")[5]
    models.append(model)
MODELS2=np.unique(models)
 
MODELS_do_c6 = intersection(MODELS, MODELS2)


In [4]:
data_dir = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip5/CONCATTED/'
data_dir_so = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_so/cmip5/'

files_c5 =  glob.glob(data_dir+ 'thetao_*yr_*_piControl*.nc')
files_so_c5 =  glob.glob(data_dir_so+ 'so_*yr_*_piControl*.nc')

models=[]
for file in files_c5:
    model = file.split("_")[5]
    models.append(model)
MODELS=np.unique(models)


models=[]
for file in files_so_c5:
    model = file.split("_")[5]
    models.append(model)
MODELS2=np.unique(models)

MODELS_do_c5 = intersection(MODELS, MODELS2)


### Below chooses a random model (CESM2) to grab a z-grid to interpolate to and 

In [6]:
file = '/gws/nopw/j04/ukfafmip/users/enewsom/regrid_from_jonathan_thetoa/cmip6/CONCATTED/thetao_Oyr_CESM2_piControl_r1i1p1f1_gn_regridded_050001-059912.nc'

dc  = xr.open_dataset(file).assign_attrs(source_id='CESM2')
dc=combined_preprocessing(dc)
Tc= dc['thetao'].isel(time=0)
ones_interp_grid = (Tc/Tc)

### Next calculate metrics salinity, density, N$^2$ (called "stratification"), or $\frac{\partial\sigma_2}{\partial y}$ and $\frac{\partial\sigma_2}{\partial z}$ for isopycal slope <br>
(Below could be written as a loop over each metric, but it takes a long time for a notebook and I liked using it interactively)

In [40]:
p=2000
averaging_period_start=0
averaging_period_end=80
g=9.8
rho_0 = 1028
lat_to_meters = 1/(111*1000)


density = 1
stratification = 1
slopes = 1

cmip6=1
cmip5=0

if cmip6:
    MODELS_do = MODELS_do_c6
    files = files_c6
    files_so = files_so_c6
    ens = 'cmip6'
else:
    MODELS_do = MODELS_do_c5
    files = files_c5
    files_so = files_so_c5
    ens='cmip5'


S=[]
ST=[]
PX=[]


for count,model in enumerate(MODELS_do[0:1]):
    print(model)
    
    files_c = [match for match in files if model in match]
    files_s = [match for match in files_so if model in match]
           
    file = files_c[0]
    file_s = files_s[0]
    
    dc  = xr.open_dataset(file).assign_attrs(source_id=model)
    dc=combined_preprocessing(dc)
    variable = list(dc.keys())[-1] #grab variable name for temp (not all the same)
    Tc= dc[variable].rename('thetao')#rename to the same variable name in data array
    Tc = Tc.isel(time = slice(averaging_period_start,averaging_period_end)).mean('time')
    Tc=Tc.where(np.abs(Tc)<1e3) #here I mask where some models have weird fill values
    Tc = Tc.rename({Tc.dims[0]:'lev'}) #here is because not all models have the same vertical coord name
    Tcx=Tc.assign_coords(model=model)
        
    ds  = xr.open_dataset(file_s).assign_attrs(source_id=model)
    ds=combined_preprocessing(ds)
    Sc= ds['so']
    Sc = Sc.isel(time = slice(averaging_period_start,averaging_period_end)).mean('time')
    Sc=Sc.where(np.abs(Tc)<1e3)
    Sc = Sc.rename({Sc.dims[0]:'lev'}) 
    Scx=Sc.assign_coords(model=model)
         
    SA = gs.SA_from_SP(Scx,p,Scx.x,Scx.y)
    CT = gs.CT_from_t(SA,Tcx,p)
    Pc = gs.density.sigma2(SA,CT)

    if stratification: 
        strat = (g/rho_0)*Pc.differentiate("lev").mean('x')
        strat = strat.assign_coords(model=model)
        strat = strat.interp_like(ones_interp_grid)
        ST.append(strat)

    if slopes:
        #slope = drhody/drhodz, but it's smoother to take zonal mean of each first so we save seperately
        drhodz = Pc.differentiate("lev").mean('x')
        drhody= Pc.differentiate("y").mean('x')*lat_to_meters
        drhodz = drhodz.assign_coords(model=model)
        drhodz = drhodz.interp_like(ones_interp_grid)
        drhody = drhody.assign_coords(model=model)
        drhody = drhody.interp_like(ones_interp_grid)
        slp = drhody/drhodz
        S.append(slp)

    if density:
        Pcx = Pc.interp_like(ones_interp_grid).mean('x')
        Pcx = Pcx.assign_coords(model=model)
        PX.append(Pcx)
  
clear_output(wait=True)
        
if slopes: 
    slope_control_model=xr.concat(S,dim='model',coords='minimal',compat='override')
    #slope_control_model.to_netcdf('slope_'+ens+'.nc')
    #slope_control_model_sig2.close()
    
if stratification:
    strat_control_model_sig2=xr.concat(ST,dim='model',coords='minimal',compat='override')
    #strat_control_model_sig2.to_netcdf('N2_'+ens+'.nc')
    #strat_control_model_sig2.close()

if density:
    Dense_control_model_sig2=xr.concat(PX,dim='model',coords='minimal',compat='override')
    #Dense_control_model_sig2.to_netcdf('density_'+ens+'.nc')
    #Dense_control_model_sig2.close()

ACCESS-CM2


/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/coding/times.py:123: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/core/indexing.py:422: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.da